# Object Detection 

Author: Siti Khotijah

Created on: 7/22/2021




# Object Detection - AutoGluon

Object detection is the process of identifying and localizing objects in an image and is an important task in computer vision. 

Our goal is to detect motorbike in images by [YOLOv3](https://pjreddie.com/media/files/papers/YOLOv3.pdf) model. A tiny dataset is collected from VOC dataset, which only contains the motorbike category. The model pretrained on the COCO dataset is used to fine-tune our small dataset. With the help of AutoGluon, we are able to try many models with different hyperparameters automatically, and return the best one as our final model.

To start, import autogluon.vision and ObjectDetector:

In [1]:
!pip install --pre autogluon 
!pip install autogluon
%matplotlib inline
import autogluon.core as ag
from autogluon.vision import ObjectDetector

     |████████████████████████████████| 268 kB 1.2 MB/s 
     |████████████████████████████████| 341 kB 7.7 MB/s 
     |████████████████████████████████| 52 kB 589 kB/s 
     |████████████████████████████████| 49 kB 3.7 MB/s 
     |████████████████████████████████| 950 kB 19.8 MB/s 
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - done
    Preparing wheel metadata ... - \ | / - \ done
     |████████████████████████████████| 206 kB 39.8 MB/s 
     |████████████████████████████████| 81 kB 5.8 MB/s 
     |████████████████████████████████| 119 kB 36.7 MB/s 
     |████████████████████████████████| 2.2 MB 41.8 MB/s 
     |████████████████████████████████| 188 kB 61.8 MB/s 
     |████████████████████████████████| 67.3 MB 63.9 MB/s 
     |████████████████████████████████| 157 kB 57.7 MB/s 
     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 2.9 MB 48.6 MB/s 
     |█████

/opt/conda/lib/python3.7/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.8.0` and `torch==1.9.0+cu102` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


# Tiny_motorbike Dataset

We collect a toy dataset for detecting motorbikes in images. From the VOC dataset, images are randomly selected for training, validation, and testing - 120 images for training, 50 images for validation, and 50 for testing. This tiny dataset follows the same format as VOC.

In [2]:
data = '../input/tiny-dataset/tiny_motorbike'
dataset_train = ObjectDetector.Dataset.from_voc(data, splits='trainval')
dataset_train

,image,rois,image_attr
0,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'motorbike', 'xmin': 0.702, 'ymin':...","{'width': 500.0, 'height': 375.0}"
1,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'motorbike', 'xmin': 0.117117117117...","{'width': 333.0, 'height': 500.0}"
2,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'person', 'xmin': 0.102, 'ymin': 0....","{'width': 500.0, 'height': 432.0}"
3,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'motorbike', 'xmin': 0.226, 'ymin':...","{'width': 500.0, 'height': 375.0}"
4,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'motorbike', 'xmin': 0.357357357357...","{'width': 333.0, 'height': 500.0}"
...,...,...,...
165,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'motorbike', 'xmin': 0.0, 'ymin': 0...","{'width': 500.0, 'height': 333.0}"
166,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'person', 'xmin': 0.408, 'ymin': 0....","{'width': 500.0, 'height': 375.0}"
167,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'person', 'xmin': 0.382, 'ymin': 0....","{'width': 500.0, 'height': 375.0}"
168,../input/tiny-dataset/tiny_motorbike/JPEGImage...,"[{'class': 'motorbike', 'xmin': 0.272, 'ymin':...","{'width': 500.0, 'height': 375.0}"


# Fit Models by AutoGluon

In this section, we demonstrate how to apply AutoGluon to fit our detection models. We use mobilenet as the backbone for the YOLOv3 model. Two different learning rates are used to fine-tune the network. The best model is the one that obtains the best performance on the validation dataset. You can also try using more networks and hyperparameters to create a larger searching space.

We fit a classifier using AutoGluon as follows. In each experiment (one trial in our searching space), we train the model for 5 epochs to avoid bursting our tutorial runtime.

In [3]:
time_limit = 60*30  # at most 0.5 hour
detector = ObjectDetector()
hyperparameters = {'epochs': 5, 'batch_size': 8}
hyperparamter_tune_kwargs={'num_trials': 2}
detector.fit(dataset_train, time_limit=time_limit, hyperparameters=hyperparameters, hyperparamter_tune_kwargs=hyperparamter_tune_kwargs)

100%|██████████| 82589/82589 [00:04<00:00, 18382.18KB/s]


9624KB [00:01, 7832.93KB/s]                           


In [4]:
dataset_test = ObjectDetector.Dataset.from_voc(data, splits='test')

test_map = detector.evaluate(dataset_test)
print("mAP on test dataset: {}".format(test_map[1][-1]))

mAP on test dataset: 0.32569292556261864


Below, we randomly select an image from test dataset and show the predicted class, box and probability over the origin image, stored in predict_class, predict_rois and predict_score columns, respectively. You can interpret predict_rois as a dict of (xmin, ymin, xmax, ymax) proportional to original image size.

In [5]:
image_path = dataset_test.iloc[0]['image']
result = detector.predict(image_path)
print(result)

   predict_class  predict_score  \
0      motorbike       0.988112   
1         person       0.970208   
2        bicycle       0.294567   
3         person       0.225033   
4         person       0.224180   
..           ...            ...   
95        person       0.032129   
96        person       0.032065   
97        person       0.032060   
98        person       0.031963   
99        person       0.031809   

                                         predict_rois  
0   {'xmin': 0.32461073994636536, 'ymin': 0.445641...  
1   {'xmin': 0.4067949950695038, 'ymin': 0.2784966...  
2   {'xmin': 0.0, 'ymin': 0.6488332152366638, 'xma...  
3   {'xmin': 0.3642578125, 'ymin': 0.2946056425571...  
4   {'xmin': 0.33951500058174133, 'ymin': 0.437841...  
..                                                ...  
95  {'xmin': 0.3480450212955475, 'ymin': 0.1806999...  
96  {'xmin': 0.7947438359260559, 'ymin': 0.3857879...  
97  {'xmin': 0.8372661471366882, 'ymin': 0.3870421...  
98  {'xmin': 0.8781

Prediction with multiple images is permitted:

In [6]:
bulk_result = detector.predict(dataset_test)
print(bulk_result)

     predict_class  predict_score  \
0        motorbike       0.988112   
1           person       0.970208   
2          bicycle       0.294567   
3           person       0.225033   
4           person       0.224180   
...            ...            ...   
4594        person       0.034571   
4595        person       0.034512   
4596        person       0.034480   
4597        person       0.034463   
4598        person       0.034428   

                                           predict_rois  \
0     {'xmin': 0.32461073994636536, 'ymin': 0.445641...   
1     {'xmin': 0.4067949950695038, 'ymin': 0.2784966...   
2     {'xmin': 0.0, 'ymin': 0.6488332152366638, 'xma...   
3     {'xmin': 0.3642578125, 'ymin': 0.2946056425571...   
4     {'xmin': 0.33951500058174133, 'ymin': 0.437841...   
...                                                 ...   
4594  {'xmin': 0.015702269971370697, 'ymin': 0.35812...   
4595  {'xmin': 0.09243986010551453, 'ymin': 0.338881...   
4596  {'xmin': 0.0142956

# Reference :
This section provides more resources on the topic if you are looking to go deeper.



* https://github.com/awslabs/autogluon